# NASA Data Exploration


In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
# Load Data
raw_data_dir = '../data/raw/'
processed_data_dir = '../data/processed/'

# - Raw Data Files
nasa_temp_file = os.path.join(raw_data_dir, 'nasa_temperature_anomaly.txt')
nasa_sea_file = os.path.join(raw_data_dir, 'nasa_sea_level.txt')
nasa_co2_file = os.path.join(raw_data_dir, 'nasa_carbon_dioxide_levels.txt')


In [ ]:
# - Process temperature data
temp_data = pd.read_csv(nasa_temp_file, sep='\t', header=None)
temp_data.columns = ['Year', 'Annual Mean', 'Lowness Smoothing']
temp_data.set_index('Year', inplace=True)

fig, ax = plt.subplots(figsize=(12, 8))

temp_data.plot(ax=ax)
ax.grid(True, linestyle='--', color='grey', alpha=0.6)

ax.set_title('Global Temperature Anomaly Data')
ax.set_ylabel('Temperature Anomaly ($\degree$C)')
ax.set_xlabel('Date')
ax.legend()

plt.show();

In [ ]:
# - Process Sea-level File
# -- Figure out header rows
with open(nasa_sea_file, 'r') as fin:
    all_lines = fin.readlines()
    
header_lines = np.array([1 for x in all_lines if x.startswith('HDR')]).sum()
sea_level_data = pd.read_csv(nasa_sea_file, delim_whitespace=True, 
                             skiprows=header_lines-1).reset_index()

sea_level_data.columns = ['Altimeter Type', 'File Cycle', 'Year Fraction', 
                          'N Observations', 'N Weighted Observations', 'GMSL',
                          'Std GMSL', 'GMSL (smoothed)', 'GMSL (GIA Applied)',
                          'Std GMSL (GIA Applied)', 'GMSL (GIA, smoothed)',
                          'GMSL (GIA, smoothed, filtered)']
sea_level_data.set_index('Year Fraction', inplace=True)

fig, ax = plt.subplots(figsize=(12, 8))

sea_level_var = sea_level_data.loc[:, 'GMSL (GIA, smoothed, filtered)'] \
                - sea_level_data.loc[:, 'GMSL (GIA, smoothed, filtered)'].iloc[0]
sea_level_var = pd.concat([sea_level_var, sea_level_data.loc[:, 'Std GMSL (GIA Applied)']], 
                          axis=1)
sea_level_var.columns = ['Value', 'Std Dev']

sea_level_var.plot(ax=ax)
ax.set_ylabel('Sea Height Change (mm)')
ax.set_xlabel('Date')
ax.legend(loc='lower right')

plt.show();

In [ ]:
# - Process Carbon Dioxide Data
with open(nasa_co2_file, 'r') as fin:
    all_lines = fin.readlines()

header_lines = np.array([1 for x in all_lines if x.startswith('#')]).sum()

co2_data = pd.read_csv(nasa_co2_file, skiprows=header_lines, header=None, 
                       delim_whitespace=True)
co2_data[co2_data == -99.99] = np.nan

co2_data.columns = ['Year', 'Month', 'Year Fraction', 'Average', 'Interpolated', 
                    'Trend', 'N Days']

co2_data.set_index(['Year', 'Month'], inplace=True)
new_idx = [datetime(x[0], x[1], 1) for x in co2_data.index]
co2_data.index = new_idx
co2_data.index.name = 'Date'

# - Plot
fig, ax = plt.subplots(figsize=(12, 8))

co2_data.loc[:, 'Average'].plot(ax=ax)

ax.set_ylabel('$CO_{2}$ Level (ppm)')
ax.set_xlabel('Date')
ax.set_title('Global $CO_{2}$ Levels over Time')

plt.show();